In [1]:
import os
import time 
import datetime
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.filterwarnings('ignore')
import numpy as np
import pandas as pd
from statsmodels.tsa.api import ExponentialSmoothing, STLForecast
from prophet import Prophet
# from models import fcast_ES_HW,fcast_rachel,fcast_prophet
from natsort import index_natsorted

### DMC2022 dataset

In [2]:
df = pd.read_excel("./archive/orders.xlsx")
df.head()

,date,userID,itemID,order
0,2020-06-01,38769,3477,1
1,2020-06-01,42535,30474,1
2,2020-06-01,42535,15833,1
3,2020-06-01,42535,20131,1
4,2020-06-01,42535,4325,1


In [3]:
df = df.dropna(how="all") # drop null value lines
df.drop_duplicates(inplace=True) # drop duplicate lines
df.sort_values(by=["date","itemID"],ignore_index=True,inplace=True)

In [4]:
df.head()

,date,userID,itemID,order
0,2020-06-01,30887,6,1
1,2020-06-01,33124,9,1
2,2020-06-01,23846,34,1
3,2020-06-01,34904,73,1
4,2020-06-01,10898,76,3


In [5]:
df.drop(["userID"],axis=1,inplace=True)

In [6]:
df.dtypes

date      datetime64[ns]
itemID             int64
order              int64
dtype: object

In [7]:
df.itemID.unique().size

31580

In [8]:
df.shape

(1048332, 3)

In [9]:
df.query('itemID == 0 ')

,date,itemID,order
229579,2020-07-25,0,1


In [10]:
df_aggr_1 = df.groupby("itemID").resample('W',on="date").sum()

In [11]:
df_aggr = df_aggr_1.drop(["itemID"],axis=1)

In [12]:
df_aggr = df_aggr.reset_index()

In [13]:
df_aggr.head()

,itemID,date,order
0,0,2020-07-26,1
1,1,2020-07-05,1
2,1,2020-07-12,1
3,1,2020-07-19,0
4,1,2020-07-26,1


In [14]:
df_stats = df_aggr.groupby('itemID',as_index=False).agg(
    Qty_min=pd.NamedAgg(column="order", aggfunc="min"),
    Order_count=pd.NamedAgg(column="order", aggfunc="count"),
    Qty_sum=pd.NamedAgg(column="order", aggfunc="sum"))
df_stats

,itemID,Qty_min,Order_count,Qty_sum
0,0,1,1,1
1,1,0,30,15
2,2,1,1,1
3,3,0,24,8
4,4,0,24,22
...,...,...,...,...
31575,32770,0,31,7
31576,32771,2,1,2
31577,32772,0,33,7
31578,32773,1,1,1


In [16]:
df_stats.query('Order_count ==35 & Qty_min>15 ')

,itemID,Qty_min,Order_count,Qty_sum
381,395,49,35,2891
613,637,44,35,3360
1664,1732,17,35,1185
1713,1782,56,35,4571
1896,1970,53,35,3993
...,...,...,...,...
30125,31265,20,35,1526
30526,31683,18,35,1707
30544,31702,29,35,3165
30650,31812,17,35,2228


In [19]:
selected_IDs = list(df_stats.query('Order_count ==35 & Qty_min>15 ').itemID.unique())


In [21]:
df_aggr[df_aggr.itemID.isin(selected_IDs)].to_csv("top78Items_aggr_weekly.csv",index=False)